In [2]:
from PIL import Image , ImageDraw, ImageFont

def calci(x1, y1, x2, y2, w, h):
        x = (x2 - x1 - w) / 2 + x1
        y = (y2 - y1 - h) / 2 + y1
        arr = [x, y]
        return arr

def template_generator(backgroung_img, title, sub_title, text_below_the_name, name_1, designation_1, organization_1, name_2, designation_2, organization_2):
    """Generates the certificate template.

    Args-
        background_img: Path of the image that is to be used as the background for certificate template
        title: Title for the certificate (eg.- "CERTIFICATE OF PARTICIPATION")
        sub_title: Subtitle within the cert (eg.- "This certificate is awarded to")
        text_below_the_name: What the cert was given for
        name_1: Name of signee #1
        designation_1: Designation of signee #1
        organization_1: Organization of signee #1
        name_2: Name of signee #2
        designation_2: Designation of signee #2
        organization_2: Organization of signee #2

    Returns-
        doc_name: Name of the PDF file containing the cert
    """
    base_image = Image.open(background_img).convert('RGB') # loads the background image for the certificate template 

    #Univ_Logo = Image.open('assets/template/logo.png') 
    #Logo_copy = Univ_Logo.copy() 
    #base_image.paste(Image2copy, (100, 150)) 

    draw = ImageDraw.Draw(base_image) # draw object allows edits and retouches to be made on PIL image template

    # text line 1: Title
    draw.text((100, 375),  title, fill='rgb(0, 0, 0)', font=ImageFont.truetype('assets/Fonts/GoogleSans-Bold-v1.27.ttf', size=100))

    # text line 2: Subtitle
    draw.text((100, 575),  sub_title, fill='rgb(50, 50, 50)', font=ImageFont.truetype('assets/Fonts/GoogleSansDisplay-Regular-v1.27.ttf', size=45))

    # text line 4: Text below the participant's name
    draw.text((100, 915),  text_below_the_name, fill='rgb(25, 25, 25)', font=ImageFont.truetype('assets/Fonts/GoogleSansDisplay-Regular-v1.27.ttf', size=45))

    # lines for signature_1
    draw.line((1200,1200, 1750,1200), fill=0, width=3)
    
    # text below the line 1
    # name_1
    name_font = ImageFont.truetype('assets/Fonts/GoogleSans-Bold-v1.27.ttf', size=45)
    w, h = draw.textsize(name_1, font=name_font)
    x, y = calci(1200,1220, 1750, 1300,w,h)
    draw.text((x, y), name_1, align='center',fill='rgb(0, 0, 0)', font=name_font)

    # designation_1
    desig_font = ImageFont.truetype('assets/Fonts/GoogleSans-Regular-v1.27.ttf', size=35)
    w, h = draw.textsize(designation_1, font=desig_font)
    x, y = calci(1200,1280, 1750, 1330,w,h)
    draw.text((x, y), designation_1, align='center',fill='rgb(0, 0, 0)', font=desig_font)

    # organization_name_1
    univ_font = ImageFont.truetype('assets/Fonts/GoogleSans-Regular-v1.27.ttf', size=35)
    w, h = draw.textsize(organization_1, font=univ_font)
    x, y = calci(1200,1320, 1750, 1370,w,h)
    draw.text((x, y), organization_1, align='center',fill='rgb(0, 0, 0)', font=univ_font)

    # lines for signature_2
    draw.line((1850,1200, 2400,1200), fill='rgb(0, 0, 0)', width=3)

    # text below the line 2
    
    # name_2
    name_font = ImageFont.truetype('assets/Fonts/GoogleSans-Bold-v1.27.ttf', size=45)
    w, h = draw.textsize(name_2, font=name_font)
    x, y = calci(1850,1220, 2400, 1300,w,h)
    draw.text((x, y), name_2, align='center',fill='rgb(0, 0, 0)', font=name_font)

    # designation_2
    desig_font = ImageFont.truetype('assets/Fonts/GoogleSans-Regular-v1.27.ttf', size=35)
    w, h = draw.textsize(designation_2, font=desig_font)
    x, y = calci(1850,1280, 2400, 1330,w,h)
    draw.text((x, y), designation_2, align='center',fill='rgb(0, 0, 0)', font=desig_font)

    # organization_name_2
    univ_font = ImageFont.truetype('assets/Fonts/GoogleSans-Regular-v1.27.ttf', size=35)
    w, h = draw.textsize(organization_2, font=univ_font)
    x, y = calci(1850,1320, 2400, 1370,w,h)
    draw.text((x, y), organization_2, align='center',fill='rgb(0, 0, 0)', font=univ_font)
    
    return base_image # returns the template image object


def certificate_generator(template, name):
    """Uses a certificate template to generate certificates for the participants in an event (in PDF format).

    Args-
        template: template image object generated by template_generator function
        name: Name of the participant

    Returns-
        doc_name: Name of the PDF file containing the cert
    """
    base_copy = template.copy() # creates a copy of the original template

    draw = ImageDraw.Draw(base_copy) # draw object allows edits and retouches to be made on PIL image template

    # name: Name of the participant
    draw.text((100, 715),  name, fill='rgb(0, 0, 0)', font=ImageFont.truetype('assets/Fonts/GoogleSans-Bold-v1.27.ttf', size=65))

    doc_name= "Generated Certificates/" + name + '.pdf' 
    base_copy.save(doc_name) # saves image as a PDF file with doc_name
    return doc_name # returns name + location of file
          

In [3]:
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_mail(name , receiver_email, subject, body, sender_email, password):
    """Automated script to send generated certificates to the participants.
    
    Args-
        name: Name of the participant
        receiver_email: Email ID of participant
        subject: Subject of the mail
        body: Body text for the email
        sender_email: Email ID of the sender (NOTE: If gmail ID, then needs permission for less secure apps enabled)
        password: Password of senders account
    
    Returns- None
    """
    doc_name= "Generated Certificates/" + name + '.pdf'

    message = MIMEMultipart() # Used to create MIME email body from scratch
    message["From"] = sender_email 
    message["To"] = receiver_email
    message["Subject"] = subject

    message.attach(MIMEText(body, "plain")) # attaches body component to the mail 

    # reading and attaching the certificate
    filename = doc_name  
    with open(filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    encoders.encode_base64(part)
    
    part.add_header("Content-Disposition", f"attachment; filename= {filename}",)

    message.attach(part)
    
    text = message.as_string()

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
    

In [4]:


# Constants
title = "CERTIFICATE OF PARTICIPATION"
sub_title = "This certificate is awarded to"
text_below_the_name = "For participating in the event 'Lorem IPSUM' organised by \nDSC XYZ University xxth of xxxxxxxx 2020"
# Signing Authority 1
name_1 = "JOHN DOE"
designation_1 = "DSC LEAD"
organization_1 = "XYZ UNIVERSITY"
# Signing Authority 2
name_2 = "JOE DOE"
designation_2 = "DSC Faculty Advisor"
organization_2 = "XYZ UNIVERSITY"

# Creating template
background_img = 'assets/template/certificate.png'
template = template_generator(background_img, title, sub_title, text_below_the_name, name_1, designation_1, organization_1, name_2, designation_2, organization_2)

# id and pass
sender_email = input("Enter your email: ")
password = input("Enter your password: ")

import pandas as pd
df = pd.read_csv("assets/data/data.csv")
for ind in df.index:
    name = df['Name'][ind]
    print(name)
    certificate_generator(template, name)
    subject = "Thank you for participating in Lorem IPSUM"
    body = "Hello "+ df['Name'][ind] +"!! \nIt was great to see you at the Lorem IPSUM \nHere is your certificate of participation."

    send_mail(df['Name'][ind], df['E-mail'][ind], subject, body, sender_email, password)

ABC XYZ
Jane Doe
